In [1381]:
from __future__ import division
import numpy as np
from pycuda.compiler import DynamicSourceModule
import pycuda.autoinit
from pycuda import gpuarray
from random import shuffle, randint
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

In [1382]:
from random import Random
size = 121
min_element = 0
max_element = 150
mas = np.random.randint(min_element, max_element, size, dtype=np.int32)

# shuffle(mas)
mas = np.int32(mas)
gpu_mas = gpuarray.to_gpu(mas)

In [1383]:
ker = DynamicSourceModule("""
__global__ void shaker_sort_ker(int *mas, int low, int high, int *depth, int current_depth) {
    cudaStream_t stream; 
    cudaStreamCreateWithFlags(&stream, cudaStreamNonBlocking);
    
    if (current_depth > *depth) {
        *depth = current_depth;
    }
    
    for (int i = low; i < high; i++) {
        if (mas[i] > mas[i + 1]) {
            int temp = mas[i];
            mas[i] = mas[i + 1];
            mas[i + 1] = temp;
        }
    }

    for (int i = high; i > low; i--) {
        if (mas[i] < mas[i - 1]) {
            int temp = mas[i];
            mas[i] = mas[i - 1];
            mas[i - 1] = temp;
        }
    }

    if (low < high) {
        shaker_sort_ker <<< 1, 1, 0, stream >>>(mas, low + 1, high - 1, depth, current_depth + 1);
    }

    cudaStreamDestroy(stream);
}
""")

krnl = ker.get_function('shaker_sort_ker')


In [1384]:
depth = np.zeros(1, dtype=np.int32)

In [1385]:
print(f'Array: {list(mas)}')
krnl(gpu_mas, np.int32(0), np.int32(mas.size - 1), cuda.InOut(depth), np.int32(1), grid=(1,1,1), block=(1,1,1))
print(f'Result: {list(gpu_mas.get())}')

# cuda.memcpy_dtoh(count, gpu_count)

print(f'Depth: {depth[0]}')

Array: [61, 58, 137, 148, 41, 121, 122, 90, 122, 20, 50, 134, 135, 24, 66, 44, 100, 63, 2, 70, 107, 148, 114, 123, 56, 1, 120, 46, 118, 63, 102, 78, 138, 5, 42, 91, 25, 118, 19, 6, 125, 38, 143, 13, 92, 1, 11, 8, 19, 38, 17, 40, 49, 42, 146, 105, 52, 59, 106, 9, 117, 109, 65, 92, 29, 23, 74, 89, 15, 46, 52, 119, 115, 82, 128, 83, 55, 82, 135, 36, 61, 114, 28, 19, 118, 142, 77, 37, 74, 53, 9, 113, 11, 80, 102, 120, 90, 55, 120, 117, 77, 137, 53, 17, 130, 111, 107, 68, 96, 144, 14, 77, 47, 40, 103, 117, 136, 129, 30, 10, 94]
Result: [1, 1, 2, 5, 6, 8, 9, 9, 10, 11, 11, 13, 14, 15, 17, 17, 19, 19, 19, 20, 23, 24, 25, 28, 29, 30, 36, 37, 38, 38, 40, 40, 41, 42, 42, 44, 46, 46, 47, 49, 50, 52, 52, 53, 53, 55, 55, 56, 58, 59, 61, 61, 63, 63, 65, 66, 68, 70, 74, 74, 77, 77, 77, 78, 80, 82, 82, 83, 89, 90, 90, 91, 92, 92, 94, 96, 100, 102, 102, 103, 105, 106, 107, 107, 109, 111, 113, 114, 114, 115, 117, 117, 117, 118, 118, 118, 119, 120, 120, 120, 121, 122, 122, 123, 125, 128, 129, 130, 134, 1